In [ ]:
import qiskit
from qiskit import QuantumCircuit, transpile, assemble, Aer
from qiskit.algorithms import Shor
from qiskit.utils import QuantumInstance
import time
import math
import matplotlib.pyplot as plt
import pandas as pd


# Cell 2: Shor's Algorithm Implementation (Simplified/High-Level)
# Note: Shor's algorithm for factoring large numbers is computationally
# very intensive even for simulators. We'll stick to small numbers.

In [ ]:
def run_shors_for_n(n_value):
    print(f"Trying to factor N={n_value} with Shor's algorithm...")
    start = time.time()

    # Rough estimate for qubits needed
    qubits_required = 0
    if n_value > 1:
        qubits_required = 2 * math.ceil(math.log2(n_value)) + n_value.bit_length() + 1
    print(f"Need about {qubits_required} qubits for N={n_value}")

    # Set up simulator
    backend = Aer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(backend, shots=1024)

    try:
        shor = Shor(N=n_value, a=2, quantum_instance=quantum_instance)
        result = shor.factor()
        factors = result.factors[0]
        print(f"Got factors for {n_value}: {factors}")
    except Exception as e:
        factors = []
        print(f"Shor's failed for N={n_value}: {e}. Maybe N's too small or simulator's struggling.")

    time_to_factor = (time.time() - start) * 1000  # Convert to ms
    return time_to_factor, qubits_required, factors

In [ ]:
rsa_moduli = [15, 21, 35] # Small moduli as requested

shor_results = []
for modulus in rsa_moduli:
    factor_time, qubits, factors = run_shors_for_n(modulus)
    shor_results.append({
        'modulus_size': modulus,
        'time_to_factor_ms': factor_time,
        'qubits_required': qubits,
        'factors': factors
    })

shor_df = pd.DataFrame(shor_results)
print("\nShor's Algorithm Simulation Results:")
print(shor_df)

In [ ]:
try:
    kyber_log_df = pd.read_csv('results/log.csv')
    avg_kyber_encrypt_time = kyber_log_df['encrypt_time_ms'].mean()
    print(f"\nAverage Kyber Encryption Time: {avg_kyber_encrypt_time:.4f} ms (from log.csv)")
except FileNotFoundError:
    avg_kyber_encrypt_time = None
    print("\nWarning: log.csv not found. Cannot compare with Kyber encryption time.")

In [ ]:
# Visualization - RSA Break Time vs Modulus Size
plt.figure(figsize=(10, 6))
plt.bar(shor_df['modulus_size'].astype(str), shor_df['time_to_factor_ms'], color='skyblue')
plt.xlabel('RSA Modulus Size (N)')
plt.ylabel('Time to Factor (ms)')
plt.title("Shor's Algorithm Simulation: Time to Factor RSA Modulus")
plt.grid(axis='y', linestyle='--')
plt.savefig('results/images/rsa_break_time.png')
plt.show()

In [ ]:
# Table Comparing to Kyber Encryption Time
comparison_data = []
for index, row in shor_df.iterrows():
    comparison_data.append({
        'Algorithm': f"RSA (N={row['modulus_size']})",
        'Time (ms)': row['time_to_factor_ms'],
        'Notes': f"Shor's Algorithm, Qubits: {row['qubits_required']}"
    })

if avg_kyber_encrypt_time is not None:
    comparison_data.append({
        'Algorithm': 'Kyber (Encryption)',
        'Time (ms)': avg_kyber_encrypt_time,
        'Notes': 'Average from PQC benchmark'
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nComparison: RSA Break Time vs Kyber Encryption Time")
print(comparison_df.to_markdown(index=False)) # For easy viewing in markdown/notebook

# Save this notebook as shors_rsa_comparison.ipynb in the quantum/ directory.
